See https://github.com/theabro/ukca/blob/master/Tutorials/UMvn13.0/notebooks/iris_toz01.ipynb

In [ ]:
!pip install -q scitools-iris

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 96.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.1/518.1 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import iris
#import iris.analysis
#import iris.quickplot as qplt
#import matplotlib.pyplot as plt
#import numpy as np
#import matplotlib.dates as mdates
#import cf_units
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This step takes several minutes. Better to load individual chunks than the whole thing. See stash codes text file.
p_file = '/content/drive/MyDrive/Documents/AI4ER/PhD/Photolysis_data/atmosa.pb1988sep'
#UKCA_data = iris.load(p_file)
UKCA_O3 = iris.load_cube(p_file,iris.AttributeConstraint(STASH='m01s50i567')) # O3 --> O(1D)

/usr/local/lib/python3.10/dist-packages/iris/fileformats/pp.py:1574: UserWarning: has_year_zero kwarg ignored for idealized calendars (always True)
  self._t1 = cftime.datetime(
/usr/local/lib/python3.10/dist-packages/iris/fileformats/pp.py:1609: UserWarning: has_year_zero kwarg ignored for idealized calendars (always True)
  self._t2 = cftime.datetime(
/usr/local/lib/python3.10/dist-packages/cf_units/__init__.py:402: UserWarning: has_year_zero kwarg ignored for idealized calendars (always True)
  dates = np.array([dt and dt.replace(microsecond=0) for dt in dates])
/usr/local/lib/python3.10/dist-packages/iris/fileformats/rules.py:353: UserWarning: Unable to create instance of HybridHeightFactory. The source data contains no field(s) for 'orography'.
  warnings.warn(msg.format(factory=factory_name))


In [ ]:
'''
print(type(UKCA_data))
print(UKCA_data.shape)
print(UKCA_data)
'''
print(type(UKCA_O3))
print(UKCA_O3.shape) # 47 t, 85 z, 144 y, 192 x
print(UKCA_O3)

<class 'iris.cube.Cube'>
(47, 85, 144, 192)
m01s50i567 / (unknown)              (time: 47; model_level_number: 85; latitude: 144; longitude: 192)
    Dimension coordinates:
        time                             x                       -             -               -
        model_level_number               -                       x             -               -
        latitude                         -                       -             x               -
        longitude                        -                       -             -               x
    Auxiliary coordinates:
        forecast_period                  x                       -             -               -
        level_height                     -                       x             -               -
        sigma                            -                       x             -               -
    Scalar coordinates:
        forecast_reference_time     1988-09-01 00:00:00
    Attributes:
        STASH            

In [ ]:
# The UKCA test suite simulates 1st and 2nd September 1988.
# The most similar time of year in the ATom data is 23rd August 2016.

# Open the .csv of all the ATom data which have already been pre-processed in the script, ATom_J_data.
ATom_file = '/content/drive/MyDrive/Documents/AI4ER/PhD/Photolysis_data/ATom_MER10_Dataset.20210613/photolysis_data.csv'
ATom_data = pd.read_csv(ATom_file, index_col=0) # shape = 138454, 49. 138454 time steps, 49 = chemicals + spatial dimensions.

(138454, 49)


In [ ]:
# Pick out just the 22nd and 23rd August indices separately.
A_day1 = ATom_data[ATom_data.index.str.contains('2016-08-22')] # shape = 1994, 49. 1994 time steps for 22/8/16, 49 = chemicals + spatial dimensions.
A_day1_O3 = A_day1[['G_LAT', 'G_LONG', 'G_ALT', 'jO3_O2_O1D_CAFS']] # shape = 1994, 4.

A_day2 = ATom_data[ATom_data.index.str.contains('2016-08-23')] # shape = 1765, 49. 1765 time steps for 23/8/16, 49 = chemicals + spatial dimensions.
A_day2_O3 = A_day2[['G_LAT', 'G_LONG', 'G_ALT', 'jO3_O2_O1D_CAFS']] # shape = 1765, 4.

# Check what the time of day range is for both.
print(A_day1_O3.index[0]) # 12:53:40
print(A_day1_O3.index[-1], '\n') # 18:51:40
print(A_day2_O3.index[0]) # 15:05:40
print(A_day2_O3.index[-1]) # 20:00:20

# Trim so that all arrays are over the same times. # 15:05:40 to 18:51:40
A_day1_O3 = A_day1_O3.loc['2016-08-22 15:05:40':'2016-08-22 18:51:40'] # shape = 1205, 4. type = pandas dataframe.
A_day2_O3 = A_day2_O3.loc['2016-08-23 15:05:40':'2016-08-23 18:51:40'] # shape = 1353, 4. type = pandas dataframe.

# Check what time zones ATom and UM are using and make them consistent.
# ATom is UTC. UM is .

2016-08-22 12:53:40
2016-08-22 18:51:40 

2016-08-23 15:05:40
2016-08-23 20:00:20
(1353, 4)


In [ ]:
# Get the UM data for the same time of day for both 1st and 2nd sep separately.
# shape = 47 t, 85 z, 144 y, 192 x. type = iris cube.
# U_day1_O3 = UKCA_O3[]  Learn how to work with iris cube indices.

# compare the 2 ATom days with each other for O3.
# compare the 2 UM days with each other for O3.
# compare one of the ATom days with one of the UM days.
# Now put all the above into functions to work with different chemicals and try with NO2.